# MLP with TensorFlow Keras

In [1]:
import tensorflow as tf # type: ignore
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import json


c:\Users\Rinaldy Adin\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv("../data/iris.csv")

In [3]:
df_x = data.iloc[:, 1:5]
df_y = data.iloc[:, 5]

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Encode target labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df_y)

# Convert integers to one-hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(df_x, onehot_encoded, test_size=0.2, random_state=42)


c:\Users\Rinaldy Adin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
with open('../models/linear.json', 'r') as f:
    json_data = json.load(f)

# Extract data from JSON
input_data = np.array(json_data['case']['input'])
target_data = np.array(json_data['case']['target'])
initial_weights = [np.array(layer) for layer in json_data['case']['initial_weights']]

# Define the model architecture
model = tf.keras.Sequential()
for i, layer in enumerate(json_data['case']['model']['layers']):
    if i == 0:
        model.add(tf.keras.layers.Dense(layer['number_of_neurons'], use_bias=True, activation=layer['activation_function']))
    else:
        model.add(tf.keras.layers.Dense(layer['number_of_neurons'], activation=layer['activation_function']))

# Set initial weights
for i, layer in enumerate(model.layers):
    weights = initial_weights[i][1:]
    biases = initial_weights[i][0]
    layer.set_weights([weights, biases])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=json_data['case']['learning_parameters']['learning_rate']),
              loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model.fit(input_data, target_data, epochs=json_data['case']['learning_parameters']['max_iteration'], batch_size=json_data['case']['learning_parameters']['batch_size'])

# Evaluate the model
final_weights = [layer.get_weights() for layer in model.layers]

# Check if final weights match the expected values
# Print final weights obtained from the model
print("Final weights:")
for i, (weights, biases) in enumerate(final_weights):
    print(f"Layer {i + 1}:")
    print("Weights:")
    print(weights)
    print("Biases:")
    print(biases)

# Print expected final weights from the JSON data
print("\nExpected final weights:")
for i, layer_weights in enumerate(json_data['expect']['final_weights']):
    print(f"Layer {i + 1}:")
    print("Weights:")
    print(layer_weights)

# Evaluate the model
test_loss = model.evaluate(input_data, target_data, verbose=0)
print("Test Loss:", test_loss)

# Calculate accuracy
predictions = model.predict(input_data, verbose=0)
accuracy = np.mean(np.equal(np.argmax(target_data, axis=1), np.argmax(predictions, axis=1)))
print("Accuracy:", accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 551ms/step - accuracy: 1.0000 - loss: 0.2217
Final weights:
Layer 1:
Weights:
[[ 0.49999896  0.2999984  -0.7999988 ]
 [ 0.1999988  -0.7000016   0.4000014 ]]
Biases:
[0.19999856 0.39999777 0.10000172]

Expected final weights:
Layer 1:
Weights:
[[0.22, 0.36, 0.11], [0.64, 0.3, -0.89], [0.28, -0.7, 0.37]]
Test Loss: [0.023332977667450905, 1.0]
Accuracy: 1.0


In [117]:
with open('../models/linear.json', 'r') as f:
    json_data = json.load(f)

# Extract data from JSON
input_size = json_data['case']['model']['input_size']
input_data = np.array(json_data['case']['input'])
target_data = np.array(json_data['case']['target'])
initial_weights = [np.array(layer) for layer in json_data['case']['initial_weights']]

# Define the model architecture
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(json_data['case']['model']['input_size'],)))

layers_json = json_data['case']['model']['layers']
for i, layer_json in enumerate(layers_json):
    layer = tf.keras.layers.Dense(units=layer_json['number_of_neurons'], activation=layer_json['activation_function'])

    if (i == 0):
        layer.build(input_shape=(input_size,))
    else:
        layer.build(input_shape=(layers_json[i-1]['number_of_neurons'],))

    weights = np.array(initial_weights[i][1:])
    bias = initial_weights[i][0]
    layer.set_weights([weights, bias])
    model.add(layer)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              loss='mean_squared_error')

model.fit(input_data, target_data, epochs=1, batch_size=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - loss: 0.2167


In [118]:
# model.summary()
model.get_weights()

[array([[ 0.52      ,  0.24000001, -0.8       ],
        [ 0.14      , -0.7866667 ,  0.46666667]], dtype=float32),
 array([0.14      , 0.31333333, 0.16666666], dtype=float32)]